# 去除表情

In [1]:
from html.parser import HTMLParser
import re


def filter_emoji(desstr, restr=''):
    # 过滤表情
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)


def data_filter(content, lower):
    # 1.去掉html标记
    html = content.strip()
    html = html.strip("\n")
    result = []
    parse = HTMLParser()
    parse.handle_data = result.append
    parse.feed(html)
    parse.close()
    content = "".join(result)

    # 2.去掉url标记
    r1 = '''http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'''
    content = re.sub(r1, '', content)

    # 3.去掉@标记与@某人的标记
    r2 = "(@.*?)+ "
    content = re.sub(r2, '', content)
    content = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", content)

    # 4.去掉一些特殊字符
    r3 = u'[’!"#$%&\'()*+,-./:;<=>?@；；：．｜～\≧▽—°❄×☝➡️❤️⭐■⏰🎂🌕👇🤔📷🎤🉑➕🥑👍📒🥚📹🆔🍊☁✨📍💝👗🚄🍀🐾🍓🐋▲🔺🔗❗🦐🐫🧜‍🏃👇➕♥♀☀●巜「」☕／↓→<=>?@⁄•ω★💊🙈☕💰😂·、…★、…【】《》『』（）？“”‘’！[\\]^_`{|}~]+'
    content = re.sub(r3, '', content)

    # 5.过滤表情
    content = filter_emoji(content, restr='')
    content = re.sub(r"\[\S+\]", "", content)  # 去除表情符号

    # 6.合并正文中过多的空格
    content = re.sub(r"\s+", " ", content)

    # 7.过滤掉\ax0
    content = content.strip().replace(u'\u3000', u'').replace(u'\xa0', u'')

    # 8.是否转换为小写
    if lower:
        content = str(content).lower()

    return content.replace(" ", "")

# 去除操作

In [4]:
import pandas

ori_path = "../before/test2"
textType = ".csv"
df = pandas.read_csv(ori_path+textType)
# 去除表情
for i in range(df.shape[0]):
    df.loc[i,'text'] = data_filter(df.loc[i,'text'],True)
after_path = ori_path+"_new"
df.to_csv(after_path+textType,index=False, encoding='utf-8')
# 去除null值
d = pandas.read_csv(after_path+textType)
d.dropna(subset=['text'],inplace=True)
end_path = after_path+"_end"
d.to_csv(end_path+textType,index=False,encoding='utf-8')
# 检验数据
e = pandas.read_csv(end_path+textType)
e

ParserError: Error tokenizing data. C error: Expected 9 fields in line 10397, saw 17


# 去除重复项

In [9]:
# 因为去重和去除表情已经在录入时处理所以只需要进行重复项去除
import time
import pandas as pd
import numpy as np

name = "test2_new_end.csv"
ori_path = "../before/"+name
df = pd.read_csv(ori_path)
newdf = df.drop_duplicates(subset=['wid'],keep="first")
new_path = "../after/"+name
newdf.to_csv(new_path,index=False,encoding='utf-8')
# 检验
d = pd.read_csv(new_path)
d

,wid,time,text,uid,like_count,username,following,followed,gender
0,4529878760164897,Thu Jul 23 15:52:00 +0800 2020,不需要这么做因为5g天线放进口罩已经足以控制你们了,3101630921,96,我姓倪别的不能说了,194,2219,m
1,4529882221249717,Thu Jul 23 16:05:44 +0800 2020,比亚迪的口罩有5g芯片，千万别用,2296948284,数据缺失,成成成小四,153,312,m
2,4529882136315032,Thu Jul 23 16:05:24 +0800 2020,5g天线我控制这些木鱼脑袋干啥呀，闲的吗,6458970173,数据缺失,不超过十个,14,17,m
3,4529881889905876,Thu Jul 23 16:04:26 +0800 2020,现在我每天在这里8买各种赛今天又收获满满cbanba典静其拍啥都有一边玩一边收米今天提线六千...,2173015011,233,月季哥哥阿,67,1553,m
4,4529878547050775,Thu Jul 23 15:51:09 +0800 2020,这还真的需要专门辟谣,1895755777,38,行者居无修,126,105,f
...,...,...,...,...,...,...,...,...,...
8355,4529402060739036,Wed Jul 22 08:17:46 +0800 2020,目前死亡率还真不算高，大概千分之六到千分之十。这要真能能引发自身免疫疾病，后遗症就太可怕了，...,7435445676,0,马谡不参军,43,22,m
8356,4529393303030767,Wed Jul 22 07:42:58 +0800 2020,今天找找红契约，撕了看看能不能改变国运,3956503775,0,三元桥式神,474,736,f
8357,4529386370370931,Wed Jul 22 07:15:25 +0800 2020,早上好,3320500013,0,请叫我厕所所长,5201,170664,f
8358,4529638494443206,Wed Jul 22 23:57:16 +0800 2020,太好了,5365388364,0,桃子82724,987,103,m
